In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install autograd

In [3]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from autograd import numpy 
from autograd import grad
import autograd.numpy as np


In [4]:
#loading dataset
wine = pd.read_csv('../input/goorm3competition/winequality-red.csv')
wine_test = pd.read_csv('../input/goorm3competition/winequality-red_test.csv')

In [5]:
#data check
wine.head()

In [6]:
wine_test.head()

In [7]:
wine.info()

In [8]:
#target인 quality가 int형이므로 category로 바꾸자.
bins = (0,5,10)
group_names = ['bad','good']
wine['quality'] = pd.cut(wine['quality'], bins = bins, labels = group_names)

In [9]:
#check quality
wine.head()

In [10]:
#category로 변환한 qualtiy를 이제 logistic regression에 적용할 수 있도록 정수로 변환한다.
label_quality = LabelEncoder()

In [11]:
#"Bad"는 0, "Good"은 1로 변환한다. 
wine['quality'] = label_quality.fit_transform(wine['quality'])

In [12]:
#quality check
wine['quality'].value_counts()

In [13]:
wine.info()

In [14]:
#seperate the dataset as response varibale and feature varibales
X = wine.drop('quality', axis = 1)
y = wine['quality']

In [15]:
#이제 데이터를 필요에 따라 나누자.
#trian:val = 9:1
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, 
                                                  random_state=42,
                                                 shuffle = False)

In [16]:
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()

In [17]:
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [18]:
y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

In [19]:
y_train

In [20]:
X_train

In [21]:
#variable selection
import seaborn
seaborn.set()

In [22]:
y_col = 'quality'
x_cols = wine.columns.drop(['quality'])
print(x_cols)
print(y_col)

In [23]:
seaborn.pairplot(data=wine, height=5, aspect=1,
                x_vars = x_cols,
                y_vars = y_col)                #quality와 상관관계를 갖는 변수는 없다. 

In [24]:
#pca를 사용해보자.
from sklearn import decomposition
pca = decomposition.PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_train_pca_DF = pd.DataFrame(data=X_train_pca )

In [25]:
pca.explained_variance_ratio_               
#pca를 사용하려면 원데이터를 80%이상 표현할 수 있는게 좋은데 너무 많은 변수가 필요하므로 
#너무 많은 변수가 필요하므로 pca는 사용하지 않는다. 

In [26]:
#data normalization : z-score

#mu and sigma 계산
mu = numpy.mean(X_train, axis = 0)
sigma = numpy.std(X_train, axis = 0)

#normalize the training, validation, and test datasets
X_train = (X_train - mu) / sigma
X_val = (X_val - mu) / sigma


In [27]:
#데이터를 이용하여 간단하게 모델을 만들어보자. 

def logistic(x):
    """The logistic function
    Arguments
    ---------
    numpy.ndarray
       The output.
    """
    out = 1. / (1. + numpy.exp(-x))
    return out


def logistic_model(x, params):   #X_train, (w,b)
    """A prediction model based on the logistic function
    
    Arguments: 
       x: array of x data. (n_samples, n_total_features)
       params: model parameters. a tuple/list of two elements
          The first element is a 1D array with shape (n_total_features). The second
          elements is a scalar (the intercept)
          
    Returns
    -------
    probabilites: numpy.adarray
       The output is a 1D array with length n_samples
    """
    f = np.dot(x, params[0]) + params[1]
    y = logistic(f)
    return y


def model_loss(x, y, params, _lambda = 1.0):
    """The logistic loss function
    Arguments:
       params: array(w, b) of model parameters
       model: the Python function for the logistic model
       x, y: arrays of input data to the model
    """
    y_pred = logistic_model(x, params)
    loss = -(
        np.dot(y, np.log(y_pred + 1e-15)) +  #regularization을 활용하여 over-fitting을 줄였다.
        np.dot((1. - y), np.log(1. - y_pred + 1e-15))) + _lambda * numpy.sum(params[0]**2)
     
    return loss

def classify(x, params):    
    """A function that determines the class.
    Arguments:
       params: array(w, b) of model parameters
       x: 1D ndarray. 
          if value>=0.5, predicted value = 1, if it's not that, predicted value = 0
    """
    probabilities = logistic_model(x, params)
    labels = (probabilities >= 0.5).astype(float)
    return labels

In [28]:
def performance(predictions, answers, beta=1.0):
    """Calculate precision, recall, and F-score.
    
    Arguments
    ---------
    predictions : numpy.ndarray of integers
        The predicted labels.
    answers : numpy.ndarray of integers
        The true labels.
    beta : float
        A coefficient representing the weight of recall.
    
    Returns
    -------
    precision, recall, score, accuracy : float
        Precision, recall, and F-score, accuracy respectively.
    """
    true_idx = (answers == 1)  # the location where the answers are 1
    false_idx = (answers == 0)  # the location where the answers are 0
    
    # true positive: answers are 1 and predictions are also 1
    n_tp = numpy.count_nonzero(predictions[true_idx] == 1)
    
    # false positive: answers are 0 but predictions are 1
    n_fp = numpy.count_nonzero(predictions[false_idx] == 1)
    
    # true negative: answers are 0 and predictions are also 0
    n_tn = numpy.count_nonzero(predictions[false_idx] == 0)
    
    # false negative: answers are 1 but predictions are 0
    n_fn = numpy.count_nonzero(predictions[true_idx] == 0)
    
    # precision, recall, and f-score
    precision = n_tp / (n_tp + n_fp+1)
    recall = n_tp / (n_tp + n_fn+1)
    score = (
        (1.0 + beta**2) * precision * recall / 
        (beta**2 * precision + recall+1)
    )

    accuracy = (n_tp + n_tn) / (n_tp + n_fn + n_fp + n_tn)

    return precision, recall, score, accuracy

In [29]:
#초기화
gradients = grad(model_loss, argnum = 2)

w = np.zeros(X_train.shape[1], dtype = float)
b = 0.

In [30]:
#초기 정확도
pred_labels_test = classify(X_train, (w,b))
perf = performance(pred_labels_test, y_train)

print("Initial precision: {:.1f}%".format(perf[0]*100))
print("Initial recall: {:.1f}%".format(perf[1]*100))
print("Initial F-score: {:.1f}%".format(perf[2]*100))
print("Initial Accuracy: {:.1f}%".format(perf[3]*100))

In [31]:
#모델을 학습시켜보자.

#learning rate
lr = 1e-5

#변동변수
change = np.inf

#a counter for optimization iterations
i = 0

#이전 반복으로 인한 유효성 검사 손실을 저장할 변수
old_val_loss = 1e-15

#running
while change >= 1e-15 and i < 10000:
    
    #기울기 계산. gradient descent 사용
    grads = gradients(X_train, y_train, (w,b))
    w -= (grads[0] * lr)
    b -= (grads[1] * lr)
    
    #validation loss
    val_loss = model_loss(X_val, y_val, (w,b))
    
    #calculate f-scores against the validation dataset
    pred_labels_val = classify(X_val, (w,b))
    score = performance(pred_labels_val, y_val)
    
    #calculate the change in validation loss
    change = np.abs((val_loss-old_val_loss)/old_val_loss)
    
    #update the counter and old_val_loss
    i += 1
    old_val_loss = val_loss
    
    #print the progress every 10 steps
    if i % 100 == 0:
        print("{}...",format(i), end = '')
        
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Validation loss:", val_loss)
print("    Validation precision:", score[0])
print("    Validation recall:", score[1])
print("    Validation F-score:", score[2])
print("    Validation Accuracy:", score[3])
print("    Change in validation loss:", change)

In [32]:
#test데이터 전처리
X_test = wine_test.to_numpy()
X_test = (X_test - mu) / sigma

In [33]:
X_test.dtype

In [34]:
X_test.shape

In [35]:
#최종 성능 구하기
pred_labels_test2 = classify(X_test, (w, b))
len(pred_labels_test2)

In [36]:
file = open('./prediction.csv','w')
file.write('ID,quality\n')
for i, pred in enumerate(pred_labels_test2):
    file.write(str(i+1)+','+str(pred)+'\n')
file.close()

In [37]:
df = pd.read_csv('prediction.csv')

In [38]:
df['quality'].value_counts()

In [45]:
df.info()

In [ ]:
df['quality'] = df['quality'].astype(int)

In [44]:
df.to_csv('prediction.csv',index=False)